In [60]:
%cleanup -f

In [1]:
%%configure -f
{
    "numExecutors": 12,
    "driverMemory": "20g"
}

In [2]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1673843097496_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1673843097496_0002,pyspark,idle,Link,Link,None,✔


In [4]:
csv_dir = 's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv'
updated_csv_dir = 's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/updated_csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Node Features

In [7]:
node_features = spark.read.csv(f'{csv_dir}/node_feature_norm/', header=True)
node_features.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+---+------------+----------+------------+----+------------+----+------------+----+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7         |f_8       |f_9         |f_10|f_11        |f_12|f_13        |f_14|f_15|f_16        |f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|f_30     |
+--------+-------+---+---+---+---+---+---+---+------------+----------+------------+----+------------+----+------------+----+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|256166  |1      |0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0032774243|0.0       |0.0         |0.0 |0.0         |0.0 |0.0018656716|0.0 |0.0 |0.0         |0.5 |0.5 |0.5 |0.5 |0.5 |0.0 |0.0 |0.0 |0.0 |0.0 |0.5 |0.5 |0.5 |0.0      |
|397382  |8      |0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0         |0.0       |0.0         |0.0 |0.0         |0.0 |6.2189053E-4|0.0 |0.0 

In [20]:
# the selected set of ids
selected_trace_ids = node_features.groupBy('trace_id').agg(
    f.sum('node_id').alias('num_nodes')
).filter('num_nodes > 4').select('trace_id')

selected_trace_ids = selected_trace_ids.withColumn(
    'new_index', f.row_number().over(Window.orderBy('trace_id')) - 1
)

selected_trace_ids.persist()
selected_trace_ids.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+
|trace_id|new_index|
+--------+---------+
|    1000|        0|
|  100000|        1|
| 1000000|        2|
|10000010|        3|
|10000011|        4|
|10000017|        5|
|10000018|        6|
|10000019|        7|
| 1000002|        8|
|10000020|        9|
|10000023|       10|
|10000024|       11|
|10000025|       12|
|10000027|       13|
|10000029|       14|
| 1000003|       15|
|10000030|       16|
|10000031|       17|
|10000032|       18|
|10000034|       19|
+--------+---------+
only showing top 20 rows

In [23]:
selected_trace_ids.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7846518

In [19]:
node_features.join(
    selected_trace_ids,
    on='trace_id',
    how='inner'
).show(5, False)  # 89052869

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+---+---+-----------+------------+------------+------------+------------+------------+----------+------------+----+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+---------+
|trace_id|node_id|f_0|f_1|f_2|f_3|f_4|f_5|        f_6|         f_7|         f_8|         f_9|        f_10|        f_11|      f_12|        f_13|f_14|f_15|        f_16|f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|      f_30|new_index|
+--------+-------+---+---+---+---+---+---+-----------+------------+------------+------------+------------+------------+----------+------------+----+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+---------+
|  256166|      1|0.0|0.0|0.0|0.0|0.0|0.0|        0.0|0.0032774243|         0.0|         0.0|         0.0|         0.0|       0.0|0.0018656716| 0.0| 0.0|         0.0| 0.5| 0.5| 0.5| 0.5| 0.5| 0.0| 0.0| 0.0| 0.0| 0.0| 0.5| 0.5| 0.5|       0.0

In [28]:
updated_node_features = node_features.join(
    selected_trace_ids,
    on='trace_id',
    how='inner'
).drop('trace_id').withColumnRenamed('new_index', 'trace_id')

updated_node_features.show(5,False)
updated_node_features.repartition(1).write.mode('overwrite').csv(
    f'{updated_csv_dir}/node_feature_norm/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+---+---+---+---+---+---+------------+----------+------------+----+------------+----+------------+----+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+--------+
|node_id|f_0|f_1|f_2|f_3|f_4|f_5|f_6|f_7         |f_8       |f_9         |f_10|f_11        |f_12|f_13        |f_14|f_15|f_16        |f_17|f_18|f_19|f_20|f_21|f_22|f_23|f_24|f_25|f_26|f_27|f_28|f_29|f_30     |trace_id|
+-------+---+---+---+---+---+---+---+------------+----------+------------+----+------------+----+------------+----+----+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+--------+
|1      |0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0032774243|0.0       |0.0         |0.0 |0.0         |0.0 |0.0018656716|0.0 |0.0 |0.0         |0.5 |0.5 |0.5 |0.5 |0.5 |0.0 |0.0 |0.0 |0.0 |0.0 |0.5 |0.5 |0.5 |0.0      |3481754 |
|8      |0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0         |0.0       |0.0         |0.0 |0.0         |0.0 |6.2189053E-4|0.0 |0.0 |0.0     

## Edge Index

In [27]:
edge_index = spark.read.csv(f'{csv_dir}/edge_index/', header=True)
edge_index.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+
|trace_id|src_id|dst_id|
+--------+------+------+
|56      |1     |2     |
|105     |0     |3     |
|141     |0     |2     |
|173     |7     |16    |
|259     |0     |6     |
+--------+------+------+
only showing top 5 rows

In [29]:
updated_edge_index = edge_index.join(
    selected_trace_ids,
    on='trace_id',
    how='inner'
).drop('trace_id').withColumnRenamed('new_index', 'trace_id')

updated_edge_index.repartition(1).write.mode('overwrite').csv(
    f'{updated_csv_dir}/edge_index/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Node Types

In [32]:
# node_types = spark.read.csv(f'{csv_dir}/node_types/', header=False)
# node_types.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
@f.udf(returnType=ArrayType(ArrayType(IntegerType())))
def construct_node_types(node_list):
    node_types = [[] for i in range(3)]
    for ntype, nid in node_list:
        node_types[int(ntype)].append(nid)
    return node_types
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
node_overview = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_overview/')
node_overview.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------------------+---------+-------+
|trace_id|node_name                    |node_type|node_id|
+--------+-----------------------------+---------+-------+
|261     |2100006525                   |0        |0      |
|261     |227444_HurstvilleCBD_L18B_2  |1        |1      |
|261     |227444_HurstvilleCBD_L18A_2  |1        |2      |
|261     |201803_Penhust594FstRd_L08A_2|1        |3      |
|261     |228290_NorthKogarah_L18A_1   |1        |4      |
+--------+-----------------------------+---------+-------+
only showing top 5 rows

In [35]:
updated_node_types = node_overview.join(
    selected_trace_ids,
    on='trace_id',
    how='inner'
).drop('trace_id').withColumnRenamed('new_index', 'trace_id').groupBy('trace_id').agg(
    construct_node_types(
        f.collect_list(f.struct('node_type', 'node_id'))
    ).alias('node_types')
).orderBy('trace_id')

# updated_node_types.count() 7846518
updated_node_types.select(
    'trace_id', f.col('node_types').cast('string')
).repartition(1).write.mode('overwrite').csv(
    f'{updated_csv_dir}/node_types/', header=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Trace Info

In [36]:
trace_info = spark.read.csv(f'{csv_dir}/trace_info/', header=True)
trace_info.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------------+----------+----------------+
|trace_id|customer_experience_index|trace_bool|error_trace_type|
+--------+-------------------------+----------+----------------+
|584395  |90.0                     |1         |1               |
|584396  |71.0                     |1         |1               |
|584397  |76.0                     |1         |1               |
|584398  |83.0                     |1         |1               |
|584399  |86.0                     |1         |1               |
+--------+-------------------------+----------+----------------+
only showing top 5 rows

In [37]:
updated_trace_info = trace_info.join(
    selected_trace_ids,
    on='trace_id',
    how='inner'
).drop('trace_id').withColumnRenamed('new_index', 'trace_id')

updated_trace_info.repartition(1).write.mode('overwrite').csv(
    f'{updated_csv_dir}/trace_info/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
updated_trace_info.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7846518

## Train/Eval/Test (small)

In [40]:
updated_trace_info.show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+----------+----------------+--------+
|customer_experience_index|trace_bool|error_trace_type|trace_id|
+-------------------------+----------+----------------+--------+
|90.0                     |1         |1               |5416926 |
|71.0                     |1         |1               |5416928 |
|86.0                     |1         |1               |5416945 |
|83.0                     |1         |1               |5416953 |
|74.0                     |1         |1               |5416969 |
+-------------------------+----------+----------------+--------+
only showing top 5 rows

In [39]:
updated_trace_info.groupBy('trace_bool').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+
|trace_bool|  count|
+----------+-------+
|         0|1811391|
|         1|6035127|
+----------+-------+

In [41]:
normal_traces = updated_trace_info.filter('trace_bool = 1')
sample_pool = normal_traces.withColumn(
    'rand', f.rand()
).withColumn(
    'row_number', f.row_number().over(
        Window.partitionBy('customer_experience_index').orderBy('rand')
    )
)
sample_pool.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+----------+----------------+--------+--------------------+----------+
|customer_experience_index|trace_bool|error_trace_type|trace_id|                rand|row_number|
+-------------------------+----------+----------------+--------+--------------------+----------+
|                     75.0|         1|               1| 4110162|2.169874164681395...|         1|
|                     75.0|         1|               1| 5441665|2.301327352238758E-6|         2|
|                     75.0|         1|               1| 1140684|6.169376423170725E-6|         3|
|                     75.0|         1|               1| 4442607|6.413018637574375E-6|         4|
|                     75.0|         1|               1| 6652163|1.470359753252825...|         5|
|                     75.0|         1|               1| 6421045|1.977907016736946E-5|         6|
|                     75.0|         1|               1| 5505876|2.188814858328225...|         7|
|                     75.0|   

In [46]:
num_sample = updated_trace_info.groupBy(
    'trace_bool', 'customer_experience_index'
).count().filter('trace_bool = 1').withColumn(
    'num_sample', (f.col('count') * 0.001).cast('int')  # 0.599715
)
num_sample.show(50)
num_sample.agg(f.sum('num_sample')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------------+------+----------+
|trace_bool|customer_experience_index| count|num_sample|
+----------+-------------------------+------+----------+
|         1|                     86.0|238755|       238|
|         1|                     82.0|225214|       225|
|         1|                     81.0|228525|       228|
|         1|                     80.0|209591|       209|
|         1|                     76.0|169867|       169|
|         1|                     91.0|230846|       230|
|         1|                     79.0|197193|       197|
|         1|                    100.0| 99381|        99|
|         1|                     77.0|180163|       180|
|         1|                     74.0|150575|       150|
|         1|                     73.0|139217|       139|
|         1|                     93.0|245218|       245|
|         1|                     96.0|204861|       204|
|         1|                     98.0|139869|       139|
|         1|                   

In [47]:
train_ids = sample_pool.join(
    num_sample.select('num_sample', 'customer_experience_index'),
    on='customer_experience_index'
).filter(f.col('row_number') <= f.col('num_sample')).select('trace_id')
train_ids.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6020

In [52]:
rest_trace = updated_trace_info.join(
    train_ids,
    on='trace_id',
    how='left_anti'
)

rest_trace.show()

rest_trace_num_sample = rest_trace.groupBy('customer_experience_index', 'trace_bool').count().withColumn(
    'num_sample', (f.col('count') * 0.0004+1).cast('int')
)
rest_trace_num_sample.show()
rest_trace_num_sample.agg(f.sum('num_sample')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------------+----------+----------------+
|trace_id|customer_experience_index|trace_bool|error_trace_type|
+--------+-------------------------+----------+----------------+
| 5416926|                     90.0|         1|               1|
| 5416928|                     71.0|         1|               1|
| 5416945|                     86.0|         1|               1|
| 5416953|                     83.0|         1|               1|
| 5416969|                     74.0|         1|               1|
| 5416975|                     69.0|         0|               0|
| 5417037|                     73.0|         1|               1|
| 5417047|                     62.0|         0|               0|
| 5417056|                     78.0|         1|               1|
| 5417058|                     56.0|         0|               0|
| 5417067|                     46.0|         0|               0|
| 5417075|                     62.0|         0|               0|
| 5417080|               

In [53]:
eval_ids = rest_trace.withColumn(
    'rand', f.rand()
).withColumn(
    'row_number', f.row_number().over(
        Window.partitionBy('customer_experience_index').orderBy('rand')
    )
).join(
    rest_trace_num_sample.select('num_sample', 'trace_bool', 'customer_experience_index'),
    on=['customer_experience_index', 'trace_bool']
).filter(f.col('row_number') <= f.col('num_sample')).select('trace_id')

eval_ids.count()
# eval_ids.groupBy('trace_bool').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3185

In [ ]:
test_ids = rest_trace.join(
    eval_ids,
    on='trace_id',
    how='left_anti'
)

# test_ids.repartition(1).write.csv(
#     's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv/test_ids')

In [ ]:
train_ids.repartition(1).write.csv(
    f'{updated_csv_dir}/train_ids/')

eval_ids.repartition(1).write.csv(
    f'{updated_csv_dir}/eval_ids/')

# Reindex All on Sampled

In [13]:
small_csv_dir = 's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/csv_small'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# train_ids.repartition(1).write.csv(
#     f'{small_csv_dir}/train_ids/')

# eval_ids.repartition(1).write.csv(
#     f'{small_csv_dir}/eval_ids/')
train_ids = spark.read.csv(f'{small_csv_dir}/train_ids/').withColumnRenamed('_c0', 'trace_id').withColumn('type', f.lit('train'))
eval_ids = spark.read.csv(f'{small_csv_dir}/eval_ids/').withColumnRenamed('_c0', 'trace_id').withColumn('type', f.lit('eval'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
small_sample_ids = train_ids.union(
    eval_ids
)

small_sample_ids = small_sample_ids.withColumn(
    'new_index', f.row_number().over(Window.orderBy('trace_id')) - 1
)
small_sample_ids.persist()
small_sample_ids.show()
small_sample_ids.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+---------+
|trace_id| type|new_index|
+--------+-----+---------+
| 1000245| eval|        0|
| 1000335|train|        1|
| 1001063| eval|        2|
| 1001304|train|        3|
| 1001533|train|        4|
| 1002410|train|        5|
| 1003860|train|        6|
| 1004017| eval|        7|
| 1004431|train|        8|
| 1005729| eval|        9|
| 1005737| eval|       10|
| 1006595| eval|       11|
| 1007014|train|       12|
| 1007269| eval|       13|
| 1007461|train|       14|
| 1009010|train|       15|
| 1009514|train|       16|
| 1011825| eval|       17|
| 1012674|train|       18|
| 1015322| eval|       19|
+--------+-----+---------+
only showing top 20 rows

9205

In [20]:
@f.udf(returnType=ArrayType(ArrayType(IntegerType())))
def construct_node_types(node_list):
    node_types = [[] for i in range(3)]
    for ntype, nid in node_list:
        node_types[int(ntype)].append(nid)
    return node_types
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def write_sample_with_reindex(input_dir, output_dir, sample_id):
    # Node Feature
    node_features = spark.read.csv(f'{input_dir}/node_feature_norm/', header=True)
    updated_node_features = node_features.join(
        sample_id,
        on='trace_id',
        how='inner'
    ).drop('trace_id').withColumnRenamed('new_index', 'trace_id')
    print(f'updated_node_features: {updated_node_features.count()}')
    updated_node_features.repartition(1).write.mode('overwrite').csv(
        f'{output_dir}/node_feature_norm/', header=True)
    
    # Edge Index
    edge_index = spark.read.csv(f'{input_dir}/edge_index/', header=True)
    updated_edge_index = edge_index.join(
        sample_id,
        on='trace_id',
        how='inner'
    ).drop('trace_id').withColumnRenamed('new_index', 'trace_id')
    print(f'updated_edge_index: {updated_edge_index.count()}')
    updated_edge_index.repartition(1).write.mode('overwrite').csv(
        f'{output_dir}/edge_index/', header=True)
    
    # Node Type
    node_overview = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset/node_overview/')
    updated_node_types = node_overview.join(
        sample_id,
        on='trace_id',
        how='inner'
    ).drop('trace_id').withColumnRenamed('new_index', 'trace_id').groupBy('trace_id').agg(
        construct_node_types(
            f.collect_list(f.struct('node_type', 'node_id'))
        ).alias('node_types')
    ).orderBy('trace_id')
    print(f'updated_node_types: {updated_node_types.count()}')
    updated_node_types.select(
        'trace_id', f.col('node_types').cast('string')
    ).repartition(1).write.mode('overwrite').csv(
        f'{output_dir}/node_types/', header=False)
    
    # Trace Info
    trace_info = spark.read.csv(f'{input_dir}/trace_info/', header=True)
    updated_trace_info = trace_info.join(
        sample_id,
        on='trace_id',
        how='inner'
    ).drop('trace_id').withColumnRenamed('new_index', 'trace_id')

    print(f'updated_trace_info: {updated_trace_info.count()}')
    updated_trace_info.repartition(1).write.mode('overwrite').csv(
        f'{output_dir}/trace_info/', header=True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
write_sample_with_reindex(
    input_dir=csv_dir,
    output_dir=small_csv_dir,
    sample_id=small_sample_ids
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

updated_node_features: 68076
updated_edge_index: 67630
updated_node_types: 9205
updated_trace_info: 9205

In [34]:
small_sample_ids.drop('trace_id').withColumnRenamed('new_index', 'trace_id').filter(
    "type == 'train'"
).select('trace_id').repartition(1).write.mode('overwrite').csv(f'{small_csv_dir}/train/')

small_sample_ids.drop('trace_id').withColumnRenamed('new_index', 'trace_id').filter(
    "type == 'eval'"
).select('trace_id').repartition(1).write.mode('overwrite').csv(f'{small_csv_dir}/eval/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…